### EDA: exploratory data analysis

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

import pandas as pd
pd.set_option('display.max_columns', 30)

import requests

In [ ]:
current_datetime = datetime.now() - relativedelta(months = 2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")

In [ ]:
url = (f"https://data.cityofchicago.org/resource/ajtu-isnz.json?"
	   f"$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' "
	   f"AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'"
	   f"&$limit=30000")
response = requests.get(url)
data = response.json()
taxi_trips = pd.DataFrame(data)

In [ ]:
taxi_trips.describe()

In [ ]:
taxi_trips[taxi_trips['fare'].isna()].sample(5)

## adattranszformációk

In [ ]:
# NaN értékek kezelése
taxi_trips = taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract'], axis = 1)
taxi_trips = taxi_trips.drop(['pickup_centroid_location', 'dropoff_centroid_location'], axis = 1)

In [ ]:
# eldobjuk azokat a sorokat, amelyekben null van
taxi_trips.dropna(inplace = True)

In [ ]:
taxi_trips.info()

In [ ]:
# oszlopok átnevezése
taxi_trips = taxi_trips.rename(
		columns = {'pickup_community_area' : 'pickup_community_area_id', 'dropoff_community_area' : 'dropoff_community_area_id'})

In [ ]:
# időjárás adatok segéd oszlop - adott trip start timestamp órára lekerekített értéke
taxi_trips['trip_start_timestamp'] = pd.to_datetime(taxi_trips['trip_start_timestamp'])
taxi_trips['datetime_for_weather'] = taxi_trips['trip_start_timestamp'].dt.floor('h')  # órára lefelé kerekít

## időjárás adatok összefűzése

In [ ]:
current_datetime = datetime.now() - relativedelta(months = 2)  # datetime típus
formatted_datetime = current_datetime.strftime("%Y-%m-%d")  # string from time formázás

url = "https://archive-api.open-meteo.com/v1/era5"

params = {
	'latitude' : 41.85,  # chicago
	'longitude' : -87.65,
	'start_date' : formatted_datetime,
	'end_date' : formatted_datetime,
	'hourly' : 'temperature_2m,wind_speed_10m,rain,precipitation'
}

response = requests.get(url, params = params)
data = response.json()

# api lekérdezés adatai metaadatok nélkül
weather_data = {
	'datetime' : data['hourly']['time'],
	'temperature' : data['hourly']['temperature_2m'],
	'wind_speed' : data['hourly']['wind_speed_10m'],
	'rain' : data['hourly']['rain'],
	'precipitation' : data['hourly']['precipitation']}

weather_df = pd.DataFrame(weather_data)
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])

In [ ]:
taxi_trips_weather = taxi_trips.merge(weather_df, left_on = 'datetime_for_weather', right_on = 'datetime')

## adattípus konverziók

In [ ]:
# eredeti verzió tárolása
taxi_trips_original = taxi_trips.copy()

In [ ]:
data_types = {
	"trip_end_timestamp" : "datetime64[ns]",
	"trip_seconds" : "int32",
	"trip_miles" : "float",
	"pickup_community_area_id" : "int8",  # -127/+127
	"dropoff_community_area_id" : "int8",
	"fare" : "float",
	"tips" : "float",
	"tolls" : "float",
	"extras" : "float",
	"trip_total" : "float"
}

taxi_trips = taxi_trips.astype(data_types)

In [ ]:
taxi_trips.describe()

#### memóriahasználat

In [ ]:
# összeadja, mennyi memóriát használ a df
orig_mem = taxi_trips_original.memory_usage(deep = True).sum()
opt_mem = taxi_trips.memory_usage(deep = True).sum()

reduction = (orig_mem - opt_mem) / orig_mem * 100

print(f'Original memory usage: {orig_mem} bytes\n'
	  f'Optimized memory usage: {opt_mem} bytes\n'
	  f'Memory reduced by: {reduction} %')

## sanity check

In [ ]:
# legutolsó trip
taxi_trips[taxi_trips['trip_end_timestamp'] == taxi_trips['trip_end_timestamp'].max()]

In [ ]:
# leghosszabb trip
taxi_trips[taxi_trips['trip_seconds'] == taxi_trips['trip_seconds'].max()]

In [ ]:
# legdrágább út
taxi_trips[taxi_trips['fare'] == taxi_trips['fare'].max()]